In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
## Read the csv file into the dataframe using pandas

data = pd.read_csv("data/data_tohoku_norm_transpose.csv", header=None)

In [5]:
time = pd.date_range("2:46PM", "6:46PM", freq="1s")

In [6]:
time

DatetimeIndex(['2017-11-13 14:46:00', '2017-11-13 14:46:01',
               '2017-11-13 14:46:02', '2017-11-13 14:46:03',
               '2017-11-13 14:46:04', '2017-11-13 14:46:05',
               '2017-11-13 14:46:06', '2017-11-13 14:46:07',
               '2017-11-13 14:46:08', '2017-11-13 14:46:09',
               ...
               '2017-11-13 18:45:51', '2017-11-13 18:45:52',
               '2017-11-13 18:45:53', '2017-11-13 18:45:54',
               '2017-11-13 18:45:55', '2017-11-13 18:45:56',
               '2017-11-13 18:45:57', '2017-11-13 18:45:58',
               '2017-11-13 18:45:59', '2017-11-13 18:46:00'],
              dtype='datetime64[ns]', length=14401, freq='S')

In [55]:
v = pd.date_range("2:46PM", "6:46PM", freq="1s")
v -= v[0]

## number of station 
station = pd.Series(range(1,439))

In [33]:
locations = pd.read_csv("data/location.txt",
                        delimiter="\t", names =["longitude", "latitude", "a", "b"])
del locations["a"], locations["b"]

In [65]:
station

0        1
1        2
2        3
3        4
4        5
5        6
6        7
7        8
8        9
9       10
10      11
11      12
12      13
13      14
14      15
15      16
16      17
17      18
18      19
19      20
20      21
21      22
22      23
23      24
24      25
25      26
26      27
27      28
28      29
29      30
      ... 
408    409
409    410
410    411
411    412
412    413
413    414
414    415
415    416
416    417
417    418
418    419
419    420
420    421
421    422
422    423
423    424
424    425
425    426
426    427
427    428
428    429
429    430
430    431
431    432
432    433
433    434
434    435
435    436
436    437
437    438
dtype: int32

In [62]:
locations['station'] =station
locations.set_index('station',inplace=True)

In [66]:
locations.loc[station]

,longitude,latitude
station,,
1,NaN,NaN
2,-98.683,27.065
3,-98.102,26.938
4,-98.068,26.463
5,-117.110,32.889
6,-107.790,32.532
7,-98.916,32.608
8,-98.079,32.573
9,-97.410,32.557


In [60]:
from haversine import haversine

tohuku_loc=[38.3,142.4]
tohuku_loc
cols = locations.columns.tolist()
cols = cols[-1:] + cols[:-1]
locations = locations[cols]
locations.loc[1]
locations.loc[1]    #locations = df[locations]


dist_data= haversine(locations.loc[1:station], tohuku_loc,miles=True)
dist_data

TypeError: '[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 24

### Creating index  in the data frame using the time.

In [12]:
data['time']=v
data.set_index('time',inplace=True)

In [13]:
data.shape

(14401, 438)

In [14]:
data.size

6307638

In [15]:
### Creation of the slider using time

In [ ]:
import ipywidgets


@ipywidgets.interact(station = (0, 437, 1))
def make_plot(station = 0):
    plt.subplot(211)
    plt.plot(array_vals[station])
    plt.xlim(0, 14000)
    plt.ylim(-1.0, 1.0)
    plt.subplot(212)
    plt.scatter(locations.iloc[station]["longitude"], locations.iloc[station]["latitude"])
    plt.xlim(-180, 180)
    plt.ylim(-90, 90)